In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import pearsonr, spearmanr

In [ ]:
# # Get subreddit leanings and political subreddits
# df_sr = pd.read_csv('../data/subreddit_scores.csv')
# subreddit_list = list(df_sr['community'])
# subreddit_id_map = {v:k for (k, v) in enumerate(subreddit_list)}
# subreddits = set(subreddit_list)
# partisan_scores = np.array(df_sr['partisan'])
# partisan_scores = (partisan_scores - np.mean(partisan_scores)) / np.std(partisan_scores)
# subreddit_leaning = np.digitize(partisan_scores, bins=np.array([-np.inf, -2, -1, 1, 2, np.inf])) - 3

# pol_subreddit_file = '../data/political_subreddits_labeled.txt'
# with open(pol_subreddit_file, 'r') as fp:
#     data = fp.read()
#     pol_subreddits = set(data.split("\n"))
# pol_subreddit_ids = set([subreddit_id_map[subreddit] for subreddit in pol_subreddits if subreddit in subreddit_id_map])
# pol_subreddits_id_list = sorted(list(pol_subreddit_ids))

# partisan_subreddits = {0:set(), -1:set(), 1:set()}
# partisan_subreddits_extr = {0:set(), -1:set(), 1:set()}
# for sr in pol_subreddits_id_list:
#     if subreddit_leaning[sr] == 0:
#         partisan_subreddits[0].add(sr)
#         partisan_subreddits_extr[0].add(sr)
#     elif subreddit_leaning[sr] > 0:
#         partisan_subreddits[1].add(sr)
#         if subreddit_leaning[sr] == 2:
#             partisan_subreddits_extr[1].add(sr)
#     elif subreddit_leaning[sr] < 0:
#         partisan_subreddits[-1].add(sr)
#         if subreddit_leaning[sr] == -2:
#             partisan_subreddits_extr[-1].add(sr)  


In [2]:
subreddit_focus = 'news'
year = 2016
month = 8
df = pd.read_csv('../data/regression_data_' + subreddit_focus + '-' + str(year) + '-' + str(month) + '-1-61d.tsv', sep='\t').dropna()
len(df)


41394

In [3]:
df.head()

,child_author_leaning_before,parent_author_leaning_before,child_cp_actv_before,child_sp_actv_before,child_np_actv_before,parent_cp_actv_before,parent_sp_actv_before,parent_np_actv_before,parent_cp_actv_after,parent_sp_actv_after,...,interaction,child_comment_length,child_comment_n_urls,child_comment_polarity,child_comment_toxicity,child_comment_severe_toxicity,parent_actv_all_before,parent_actv_all_after,nested,child_comment_political
0,-0.428571,1.806452,0.0,3.0,4.0,0.0,28.0,3.0,0.0,27.0,...,1,195,0,0.8183,0.654470,0.169603,0 3 28 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,0 2 27 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,1,0.8426
1,-0.357143,-0.727273,0.0,5.0,9.0,0.0,6.0,5.0,0.0,0.0,...,1,717,0,0.9338,0.044636,0.001197,4 5 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1,0.9767
2,-0.500000,-0.833333,0.0,1.0,1.0,3.0,61.0,2.0,1.0,68.0,...,1,150,0,0.9607,0.637829,0.040387,61 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,68 6 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,1,0.5635
3,0.213415,-0.842105,68.0,55.0,41.0,0.0,16.0,3.0,0.0,13.0,...,1,119,0,-0.7364,0.682712,0.043077,14 3 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0...,7 7 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1,0.7379
4,-0.033835,0.800000,0.0,9.0,257.0,0.0,2.0,3.0,3.0,2.0,...,1,151,0,-0.3400,0.025085,0.001259,0 3 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,3 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,1,0.1974


In [4]:
df = df[(df['child_comment_political'] > 0.5)]
len(df)

19002

In [5]:
# df = df[df['parent_sp_actv_after'] + df['parent_cp_actv_after'] > 0]
# len(df)

In [6]:
# Binarize terms
df['parent_author_left'] = (df['parent_author_leaning_before'] < 0).astype(int)
df['child_author_left'] = (df['child_author_leaning_before'] < 0).astype(int)
df['cross_party'] = (((df['parent_author_leaning_before'] < 0) & (df['child_author_leaning_before'] > 0)) | ((df['parent_author_leaning_before'] > 0) & (df['child_author_leaning_before'] < 0))).astype(int)
df['same_party'] = (((df['parent_author_leaning_before'] < 0) & (df['child_author_leaning_before'] < 0)) | ((df['parent_author_leaning_before'] > 0) & (df['child_author_leaning_before'] > 0))).astype(int)

df['parent_author_extreme'] = ((df['parent_author_leaning_before'] == -2) | (df['parent_author_leaning_before'] == 2)).astype(int)
df['child_author_extreme'] = ((df['child_author_leaning_before'] == -2) | (df['child_author_leaning_before'] == 2)).astype(int)


In [7]:
n_left = len(df[df['parent_author_leaning_before'] < 0])
n_right = len(df[df['parent_author_leaning_before'] > 0])
print('All users: %d left-wing, %d right-wing, ratio %.2f' % (n_left, n_right, n_left / n_right))

n_left = len(df[(df['parent_author_leaning_before'] < 0) & (df['interaction'] == 0)])
n_right = len(df[(df['parent_author_leaning_before'] > 0) & (df['interaction'] == 0)])
print('Users receiving no reply: %d left-wing, %d right-wing, ratio %.2f' % (n_left, n_right, n_left / n_right))

n_left = len(df[(df['parent_author_leaning_before'] < 0) & (df['interaction'] == 1) & (df['cross_party'] == 1)])
n_right = len(df[(df['parent_author_leaning_before'] > 0) & (df['interaction'] == 1) & (df['cross_party'] == 1)])
print('Users receiving cross-party reply: %d left-wing, %d right-wing, ratio %.2f' % (n_left, n_right, n_left / n_right))

n_left = len(df[(df['parent_author_leaning_before'] < 0) & (df['interaction'] == 1) & (df['same_party'] == 1)])
n_right = len(df[(df['parent_author_leaning_before'] > 0) & (df['interaction'] == 1) & (df['same_party'] == 1)])
print('Users receiving same-party reply: %d left-wing, %d right-wing, ratio %.2f' % (n_left, n_right, n_left / n_right))


All users: 11184 left-wing, 7818 right-wing, ratio 1.43
Users receiving no reply: 4852 left-wing, 3867 right-wing, ratio 1.25
Users receiving cross-party reply: 2316 left-wing, 2551 right-wing, ratio 0.91
Users receiving same-party reply: 4016 left-wing, 1400 right-wing, ratio 2.87


In [8]:
df['conflict'] = ((df['child_comment_score'] - 1) * (df['parent_comment_score'] - 1) < 0).astype(int)

# Take logarithm
df['log_child_score'] = np.log10(np.maximum(df['child_comment_score'], 1))

df['log_pa_cp_actv_before'] = np.log10(np.maximum(df['parent_cp_actv_before'], 1))
df['log_pa_sp_actv_before'] = np.log10(np.maximum(df['parent_sp_actv_before'], 1))
df['log_pa_np_actv_before'] = np.log10(np.maximum(df['parent_np_actv_before'], 1))
df['log_ca_cp_actv_before'] = np.log10(np.maximum(df['child_cp_actv_before'], 1))
df['log_ca_sp_actv_before'] = np.log10(np.maximum(df['child_sp_actv_before'], 1))
df['log_ca_np_actv_before'] = np.log10(np.maximum(df['child_np_actv_before'], 1))

# Normalization
for feature in ['log_child_score', 'log_pa_cp_actv_before', 'log_pa_sp_actv_before', 'log_pa_np_actv_before', 'log_ca_cp_actv_before', 'log_ca_sp_actv_before', 'log_ca_np_actv_before']:
    df[feature] = (df[feature] - df[feature].mean()) / (2 * df[feature].std(ddof=0))
    

In [9]:
# Interaction terms
df['interaction_conflict'] = df['interaction'] * df['conflict']
df['interaction_log_child_score'] = df['interaction'] * df['log_child_score']
df['interaction_nested'] = df['interaction'] * df['nested']
df['interaction_nonnested'] = df['interaction'] * (1 - df['nested'])

df['cp_interaction'] = df['cross_party'] * df['interaction']
df['sp_interaction'] = df['same_party'] * df['interaction']
df['cp_interaction_conflict'] = df['cp_interaction'] * df['conflict']
df['sp_interaction_conflict'] = df['sp_interaction'] * df['conflict']

df['cp_interaction_nested'] = df['cp_interaction'] * df['nested']
df['sp_interaction_nested'] = df['sp_interaction'] * df['nested']
df['cp_interaction_nonnested'] = df['cp_interaction'] * (1 - df['nested'])
df['sp_interaction_nonnested'] = df['sp_interaction'] * (1 - df['nested'])

df['cp_interaction_pa_left'] = df['cross_party'] * df['interaction'] * df['parent_author_left']
df['sp_interaction_pa_left'] = df['same_party'] * df['interaction'] * df['parent_author_left']
df['cp_interaction_pa_extreme'] = df['cross_party'] * df['interaction'] * df['parent_author_extreme']
df['sp_interaction_pa_extreme'] = df['same_party'] * df['interaction'] * df['parent_author_extreme']
df['cp_interaction_ca_extreme'] = df['cross_party'] * df['interaction'] * df['child_author_extreme']
df['sp_interaction_ca_extreme'] = df['same_party'] * df['interaction'] * df['child_author_extreme']


In [10]:
# Outcome variable
df['more_cp_actv'] = (df['parent_cp_actv_after'] - df['parent_cp_actv_before'] > 0).astype(float)
df['more_sp_actv'] = (df['parent_sp_actv_after'] - df['parent_sp_actv_before'] > 0).astype(float)
df['more_np_actv'] = (df['parent_np_actv_after'] - df['parent_np_actv_before'] > 0).astype(float)
df['more_cp_sp_ratio'] = ((df['parent_cp_actv_after'])/(df['parent_sp_actv_after'] + df['parent_cp_actv_after']) - (df['parent_cp_actv_before'])/(df['parent_sp_actv_before'] + df['parent_cp_actv_before']) > 0).astype(float)


In [11]:
# Multicollinearity test (VIF)
indp_vars = ['parent_author_left', 
             'parent_author_extreme', 
             'log_pa_cp_actv_before', 
             'log_pa_sp_actv_before', 
             'log_pa_np_actv_before',
             # 'interaction_nonnested',
             # 'interaction_nested',            
             'sp_interaction_nonnested',
             'sp_interaction_nested',
             'cp_interaction_nonnested',
             'cp_interaction_nested',
             ]
vif_data = df[indp_vars]
vif_res = pd.DataFrame()
vif_res['feature'] = indp_vars
vif_res['VIF'] = [variance_inflation_factor(vif_data.values, i) for i in range(len(indp_vars))]
vif_res

,feature,VIF
0,parent_author_left,1.650496
1,parent_author_extreme,1.089052
2,log_pa_cp_actv_before,1.306971
3,log_pa_sp_actv_before,1.405115
4,log_pa_np_actv_before,1.325820
5,sp_interaction_nonnested,1.096649
6,sp_interaction_nested,1.422531
7,cp_interaction_nonnested,1.028424
8,cp_interaction_nested,1.211547


In [12]:
res = sm.logit(formula=
               # 'more_cp_sp_ratio ~ '
               'more_cp_actv ~ '
               'log_pa_cp_actv_before + '
               'log_pa_sp_actv_before + '
               'log_pa_np_actv_before + '
               'parent_author_left + '
               'parent_author_extreme + '
               # 'interaction_nested + '
               # 'interaction_nonnested + '
               'sp_interaction_nonnested + '
               'sp_interaction_nested + ' 
               'cp_interaction_nonnested + '
               'cp_interaction_nested'             
               , data=df).fit()

Optimization terminated successfully.
         Current function value: 0.506165
         Iterations 5


In [13]:
K = len(res.params)
adj_pseudo_r2 = 1 - (res.llf - K) / res.llnull
adj_pseudo_r2

0.015015063496823688

In [14]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           more_cp_actv   No. Observations:                19002
Model:                          Logit   Df Residuals:                    18992
Method:                           MLE   Df Model:                            9
Date:                Sat, 05 Oct 2024   Pseudo R-squ.:                 0.01604
Time:                        00:52:18   Log-Likelihood:                -9618.1
converged:                       True   LL-Null:                       -9774.9
Covariance Type:            nonrobust   LLR p-value:                 3.489e-62
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.1266      0.040    -27.911      0.000      -1.206      -1.048
log_pa_cp_actv_before        0.1441      0.038      3.810      0.000       0.070       0.218
log_pa_sp_actv_before        0.2213      0.042      5.278      0.000       0.139       0.303
log_pa_np_actv_before       -0.0364      0.041     -0.885      0.376      -0.117       0.044
parent_author_left          -0.5055      0.040    -12.712      0.000      -0.583      -0.428
parent_author_extreme       -0.1533      0.062     -2.463      0.014      -0.275      -0.031
sp_interaction_nonnested     0.1474      0.082      1.788      0.074      -0.014       0.309
sp_interaction_nested        0.2142      0.050      4.267      0.000       0.116       0.313
cp_interaction_nonnested     0.1996      0.086      2.327      0.020       0.031       0.368
cp_interaction_nested        0.1289      0.051      2.519      0.012       0.029       0.229
============================================================================================
"""

In [15]:
res = sm.logit(formula=
               'more_sp_actv ~ '
               'log_pa_cp_actv_before + '
               'log_pa_sp_actv_before + '
               'log_pa_np_actv_before + '
               'parent_author_left + '
               'parent_author_extreme + '
               # 'interaction_nonnested + '
               # 'interaction_nested + '
               'sp_interaction_nonnested + '
               'sp_interaction_nested + ' 
               'cp_interaction_nonnested + '
               'cp_interaction_nested'
               , data=df).fit()

Optimization terminated successfully.
         Current function value: 0.601472
         Iterations 5


In [16]:
K = len(res.params)
adj_pseudo_r2 = 1 - (res.llf - K) / res.llnull
adj_pseudo_r2

0.0059057724963772484

In [17]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           more_sp_actv   No. Observations:                19002
Model:                          Logit   Df Residuals:                    18992
Method:                           MLE   Df Model:                            9
Date:                Sat, 05 Oct 2024   Pseudo R-squ.:                0.006775
Time:                        00:52:18   Log-Likelihood:                -11429.
converged:                       True   LL-Null:                       -11507.
Covariance Type:            nonrobust   LLR p-value:                 5.231e-29
============================================================================================
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                   -1.0674      0.038    -28.285      0.000      -1.141      -0.993
log_pa_cp_actv_before       -0.1731      0.037     -4.624      0.000      -0.246      -0.100
log_pa_sp_actv_before        0.2286      0.037      6.130      0.000       0.155       0.302
log_pa_np_actv_before       -0.1071      0.037     -2.880      0.004      -0.180      -0.034
parent_author_left           0.0221      0.036      0.613      0.540      -0.049       0.093
parent_author_extreme       -0.0633      0.057     -1.120      0.263      -0.174       0.047
sp_interaction_nonnested     0.2535      0.072      3.502      0.000       0.112       0.395
sp_interaction_nested        0.3268      0.044      7.366      0.000       0.240       0.414
cp_interaction_nonnested     0.2690      0.079      3.397      0.001       0.114       0.424
cp_interaction_nested        0.3626      0.046      7.955      0.000       0.273       0.452
============================================================================================
"""